In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from tqdm import tqdm

In [2]:
tmax = xr.open_dataset('data/esri/MaxTemp_2006_2021_Cali.nc')

In [8]:
tmax

<xarray.Dataset>
Dimensions:                                              (locations: 8991,
                                                          time: 5844)
Coordinates:
    lat                                                  (locations) float64 ...
    lon                                                  (locations) float64 ...
  * time                                                 (time) datetime64[ns] ...
Dimensions without coordinates: locations
Data variables: (12/13)
    projection                                           int32 ...
    x                                                    (locations) float64 ...
    y                                                    (locations) float64 ...
    poly_breaks                                          (locations) int32 ...
    poly_coords                                          (locations) float64 ...
    time_step_ID                                         (time, locations) float64 ...
    ...                                                   ...
    FIPS                                                 (time, locations) float64 ...
    MAX_TEMPERATURE_NONE_SPATIAL_NEIGHBORS               (time, locations) float64 ...
    MAX_TEMPERATURE_NONE_SPATIAL_NEIGHBORS_TREND_ZSCORE  (locations) float64 ...
    MAX_TEMPERATURE_NONE_SPATIAL_NEIGHBORS_TREND_PVALUE  (locations) float64 ...
    MAX_TEMPERATURE_NONE_SPATIAL_NEIGHBORS_TREND_BIN     (locations) float64 ...
    PREDICTION_BINARY_MASK                               (time, locations) float64 ...
Attributes: (12/28)
    subsetType:                 SUBSET
    geometry_unit:              METER
    convert_factor:             1.0
    alignment:                  END_TIME
    reference_time:             
    time_size:                  1
    ...                         ...
    esri_pe_string:             PROJCS["USA_Contiguous_Albers_Equal_Area_Coni...
    projection_authority_code:  102003
    raw_pe_string:              PROJCS["USA_Contiguous_Albers_Equal_Area_Coni...
    location_id_field:          FIPS
    rate_info:                  
    agg_shape_type:             POLYGON

In [3]:
df_tmax = tmax[['FIPS','MAX_TEMPERATURE_NONE_SPATIAL_NEIGHBORS']].to_dataframe()

In [19]:
df_tmax = df_tmax.drop(columns=['lat', 'lon'])

In [20]:
df_tmax.head()

FIPS  MAX_TEMPERATURE_NONE_SPATIAL_NEIGHBORS
time       locations                                                
2005-12-31 0          1765.0                               13.350000
           1          1766.0                               14.950006
           2          1767.0                               14.950006
           3          1768.0                               14.950006
           4          1769.0                               14.950006

In [21]:
tmin = xr.open_dataset('data/esri/MinTemp_2006_2021_Cali.nc')

In [22]:
df_tmin = tmin[['FIPS','MIN_TEMPERATURE_NONE_SPATIAL_NEIGHBORS']].to_dataframe()

In [23]:
df_tmin = df_tmin.drop(columns=['lat', 'lon'])

In [24]:
df_tmin.head()

FIPS  MIN_TEMPERATURE_NONE_SPATIAL_NEIGHBORS
time       locations                                                
2005-12-31 0          1765.0                                7.850000
           1          1766.0                                8.249994
           2          1767.0                                8.249994
           3          1768.0                                8.249994
           4          1769.0                                8.249994

In [25]:
len(df_tmax)

52543404

In [26]:
len(df_tmin)

52543404

## Join tmin and tmax

In [27]:
tqdm.pandas()
df_t = df_tmax.join(df_tmin, rsuffix= "_tmin").progress_apply(lambda x: x)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.25it/s]


In [28]:
len(df_t)

52543404

In [29]:
df_t.head()

FIPS  MAX_TEMPERATURE_NONE_SPATIAL_NEIGHBORS  \
time       locations                                                   
2005-12-31 0          1765.0                               13.350000   
           1          1766.0                               14.950006   
           2          1767.0                               14.950006   
           3          1768.0                               14.950006   
           4          1769.0                               14.950006   

                      FIPS_tmin  MIN_TEMPERATURE_NONE_SPATIAL_NEIGHBORS  
time       locations                                                     
2005-12-31 0             1765.0                                7.850000  
           1             1766.0                                8.249994  
           2             1767.0                                8.249994  
           3             1768.0                                8.249994  
           4             1769.0                                8.249994

In [30]:
# check if FIPS are the same

df_t['FIPS'].equals(df_t['FIPS_tmin'])  # Returns True

True

In [31]:
df_t = df_t.drop(columns=['FIPS'])

In [32]:
import gc

del tmin
del tmax
del df_tmin
del df_tmax
gc.collect()

2314

## Load PM2.5

In [9]:
pm25 = xr.open_dataset('data/esri/PM25_Nature_2006_2020_Cali.nc')

In [18]:
pm25["location_ID"].to_dataframe().head()

lat         lon  location_ID
time       locations                                    
2005-12-31 0          37.867656 -122.231882          0.0
           1          37.848138 -122.249591          1.0
           2          37.840584 -122.254478          2.0
           3          37.848284 -122.257445          3.0
           4          37.848545 -122.264736          4.0

In [34]:
df_pm25 = pm25[['FIPS','MEAN_NONE_SPATIAL_NEIGHBORS']].to_dataframe()

In [35]:
df_pm25 = df_pm25.drop(columns=['lat', 'lon'])

In [36]:
df_pm25.head()

FIPS  MEAN_NONE_SPATIAL_NEIGHBORS
time       locations                                     
2005-12-31 0          1765.0                     4.986000
           1          1766.0                     5.500000
           2          1767.0                     5.486667
           3          1768.0                     5.437778
           4          1769.0                     5.480000

In [37]:
len(df_pm25)

47929050

In [38]:
len(df_t)

52543404

## Test for joins

Look for fastest join

In [55]:
temp1 = df_t.head(30000)
temp2 = df_pm25.head(30000)

In [56]:
%%timeit 
temp1.join(temp2)

6.39 ms ± 79.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [57]:
%%timeit 
pd.merge(temp1, temp2, left_index=True, right_index=True, how='inner')

16.7 ms ± 19 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [58]:
%%timeit
pd.concat([temp1, temp2], axis=1)

34.6 ms ± 83.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Join between temperatures and PM25

In [48]:
df_t = df_t.rename(columns={"FIPS_tmin":"FIPS"})

In [53]:
len(df_t)

52543404

In [54]:
df_t = df_t.reset_index().set_index(["time","FIPS"])

In [55]:
df_t = df_t.drop(columns=['locations'])

In [56]:
df_t.head()

MAX_TEMPERATURE_NONE_SPATIAL_NEIGHBORS  \
time       FIPS                                             
2005-12-31 1765.0                               13.350000   
           1766.0                               14.950006   
           1767.0                               14.950006   
           1768.0                               14.950006   
           1769.0                               14.950006   

                   MIN_TEMPERATURE_NONE_SPATIAL_NEIGHBORS  
time       FIPS                                            
2005-12-31 1765.0                                7.850000  
           1766.0                                8.249994  
           1767.0                                8.249994  
           1768.0                                8.249994  
           1769.0                                8.249994

In [57]:
df_pm25 = df_pm25.reset_index().set_index(["time","FIPS"])

In [58]:
df_pm25 = df_pm25.drop(columns=['locations'])

In [59]:
df_pm25.head()

MEAN_NONE_SPATIAL_NEIGHBORS
time       FIPS                               
2005-12-31 1765.0                     4.986000
           1766.0                     5.500000
           1767.0                     5.486667
           1768.0                     5.437778
           1769.0                     5.480000

In [60]:
df = df_t.join(df_pm25).progress_apply(lambda x: x)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.80it/s]


In [61]:
df.head()

MAX_TEMPERATURE_NONE_SPATIAL_NEIGHBORS  \
time       FIPS                                             
2005-12-31 1765.0                               13.350000   
           1766.0                               14.950006   
           1767.0                               14.950006   
           1768.0                               14.950006   
           1769.0                               14.950006   

                   MIN_TEMPERATURE_NONE_SPATIAL_NEIGHBORS  \
time       FIPS                                             
2005-12-31 1765.0                                7.850000   
           1766.0                                8.249994   
           1767.0                                8.249994   
           1768.0                                8.249994   
           1769.0                                8.249994   

                   MEAN_NONE_SPATIAL_NEIGHBORS  
time       FIPS                                 
2005-12-31 1765.0                     4.986000  
           1766.0                     5.500000  
           1767.0                     5.486667  
           1768.0                     5.437778  
           1769.0                     5.480000

In [62]:
df = df.rename(columns={"MAX_TEMPERATURE_NONE_SPATIAL_NEIGHBORS":"tmax",
"MIN_TEMPERATURE_NONE_SPATIAL_NEIGHBORS":"tmin",
"MEAN_NONE_SPATIAL_NEIGHBORS":"pm25"})

In [63]:
df.to_parquet("outputs/esri_tmin_tmax_pm25_merged.parquet")

In [2]:
df = pd.read_parquet("outputs/esri_tmin_tmax_pm25_merged.parquet")

## Join wildfire day

In [3]:
wf = xr.open_dataset('data/esri/WildfirePresence_2006_2021_CA_daily.nc')

In [18]:
wf["FIPS"].to_dataframe().head()

lat         lon  FIPS
time       locations                             
2006-01-01 0          37.867656 -122.231882   0.0
           1          37.848138 -122.249591   1.0
           2          37.840584 -122.254478   2.0
           3          37.848284 -122.257445   3.0
           4          37.848545 -122.264736   4.0

In [9]:
wf["location_label"].to_dataframe().head()

,location_label
labelDim,
0,06001400100
1,06001400200
2,06001400300
3,06001400400
4,06001400500


In [ ]:
df_wf = wf[['FIPS', 'location_label', 'FIREDAY_NONE_ZEROS']].to_dataframe()

In [9]:
df_wf = df_wf.drop(columns=['lat', 'lon'])

In [8]:
df_wf

FIPS  FIREDAY_NONE_ZEROS        lat         lon
time       locations                                                   
2006-01-01 0             0.0                 0.0  37.867656 -122.231882
           1             1.0                 0.0  37.848138 -122.249591
           2             2.0                 0.0  37.840584 -122.254478
           3             3.0                 0.0  37.848284 -122.257445
           4             4.0                 0.0  37.848545 -122.264736
...                      ...                 ...        ...         ...
2021-12-29 9091       9091.0                 0.0  39.050329 -121.546676
           9092       9092.0                 0.0  39.250249 -121.520640
           9093       9093.0                 0.0  39.291833 -121.329852
           9094       9094.0                 0.0  39.465359 -121.167910
           9095       9095.0                 0.0  39.367748 -121.240208

[53138832 rows x 4 columns]

In [11]:
df_wf = df_wf.reset_index().set_index(["time","FIPS"])

In [12]:
df_wf = df_wf.drop(columns=['locations'])

In [13]:
len(df)

52543404

In [14]:
len(df_wf)

53138832

In [16]:
df = df.join(df_wf)

In [ ]:
df.to_parquet("outputs/esri_tmin_tmax_pm25_wf_merged.parquet")

## Join smoke PM2.5

In [9]:
df = pd.read_parquet("outputs/esri_tmin_tmax_pm25_wf_merged.parquet")

In [10]:
smoke_df = pd.read_parquet("outputs/smoke_pm25_predicted_with_fips.parquet")

In [11]:
df.head()

tmax      tmin      pm25  FIREDAY_NONE_ZEROS
time       FIPS                                                     
2005-12-31 1765.0  13.350000  7.850000  4.986000                 NaN
           1766.0  14.950006  8.249994  5.500000                 NaN
           1767.0  14.950006  8.249994  5.486667                 NaN
           1768.0  14.950006  8.249994  5.437778                 NaN
           1769.0  14.950006  8.249994  5.480000                 NaN

In [12]:
len(df)

52543404

In [13]:
len(smoke_df)

2599492

In [16]:
len(smoke_df)

2538869

In [15]:
smoke_df = smoke_df.groupby(["time","FIPS"]).first()

In [17]:
smoke_df.head()

GEOID  smokePM_pred
time       FIPS                            
2006-01-03 2677.0  6025010101      2.294267
           2678.0  6025010102      2.345547
           2679.0  6025010200      2.481742
           2680.0  6025010300      1.667545
           2681.0  6025010500      1.735179

In [19]:
merged = df.join(smoke_df)

In [21]:
len(merged)

52543404

In [23]:
merged = merged.drop(columns="GEOID")

In [24]:
merged.to_parquet("outputs/esri_tmin_tmax_pm25_wf_smokePM_pred_merged.parquet")